# Colab for Creating an Embedding Customization Pipeline

Requirements:
- kfp>=2.0.0b16
- google-cloud-aiplatform>=1.15

In [3]:
# !pip install google-cloud-aiplatform --upgrade

In [4]:
from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

Vertex AI SDK version: 1.26dev20230530+language_models_eval


In [5]:
# @title ## Authenticate. { display-mode: "form"}

from google.cloud import aiplatform
# from google.colab import auth as google_auth
import numpy as np
import pandas as pd
import tensorflow as tf

# google_auth.authenticate_user()

2023-11-28 04:36:21.154483: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 04:36:21.952205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 04:36:21.952307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 04:36:22.100817: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 04:36:22.395631: I tensorflow/core/platform/cpu_feature_guar

### Configuration

In [6]:
!gsutil ls 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/'

gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/test.tsv
gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/train.tsv


In [7]:
# @title ## Creating Pipeline. { display-mode: "form"}
# @markdown Creating pipeline job from the pipeline template published to KFP artifact repository. { display-mode: "form"}
MODEL_NAME = "text-bison@001" # @param {type:"string"}

#gs://my-project-0004-bucket/

VERTEX_API_PROJECT = PROJECT_ID = "my-project-0004-346516" #'your-project' #@param {"type": "string"}
VERTEX_API_LOCATION =REGION= LOCATION= 'us-central1' #'asia-southeast1' #@param {"type": "string"}

BUCKET_NAME = 'my-project-0004-bucket' # @param {type:"string"}


# @markdown Update the bucket name to reflect the source of your data. The service account `service-<project_number>@gcp-sa-aiplatform.iam.gserviceaccount.com` must have read and write objects to the GCS buckets. See the documentation for more information about the [Vertex service agents](https://cloud.google.com/vertex-ai/docs/general/access-control#service-agents) and [Cloud Storage permissions](https://cloud.google.com/vertex-ai/docs/general/access-control#storage-roles).
QUERIES_PATH = 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/queries.jsonl'  # @param {type:"string"}
CORPUS_PATH = 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/corpus.jsonl'  # @param {type:"string"}
TRAIN_LABEL_PATH = 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/train.tsv'  # @param {type:"string"}
PIPELINE_ROOT = 'gs://my-project-0004-bucket'  # @param {type:"string"}
# @markdown The validation label path is optional and if not specified the training data will be randomly split so 20% is used for validation.
VALIDATION_LABEL_PATH = ''  # @param {type:"string"}
# @markdown The test label path is optional and if not specified the training data will be randomly split so 30% is used for testing.
TEST_LABEL_PATH = 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/test.tsv'  # @param {type:"string"}

# @markdown The following parameters can generally be left at their default setting.
BATCH_SIZE = 64  # @param {type:"integer"}
ITERATIONS = 100  # @param {type:"integer"}
ACCELERATOR_TYPE = 'NVIDIA_TESLA_T4'  # @param {type:"string"}
MACHINE_TYPE = 'n1-standard-16'  # @param {type:"string"}
# MACHINE_TYPE = 'a2-highgpu-1g' #"a2-highgpu-1g"
# ACCELERATOR_TYPE = "NVIDIA_TESLA_A100"
ACCELERATOR_COUNT = 1

PARAMS = {
    'project': PROJECT_ID,
    'location': LOCATION,
    'queries_path': QUERIES_PATH,
    'corpus_path': CORPUS_PATH,
    'train_label_path': TRAIN_LABEL_PATH,
    'batch_size': BATCH_SIZE,
    'iterations': ITERATIONS,
    'accelerator_type': ACCELERATOR_TYPE,
    'machine_type': MACHINE_TYPE,
    # 'accelerator_count' : ACCELERATOR_COUNT,
}
if VALIDATION_LABEL_PATH:
  PARAMS['validation_label_path'] = VALIDATION_LABEL_PATH
if TEST_LABEL_PATH:
  PARAMS['test_label_path'] = TEST_LABEL_PATH

# template_uri = 'https://us-kfp.pkg.dev/ml-pipeline/llm-text-embedding/custom-embedding-finetuning-pipeline/preview'
template_uri = "https://us-kfp.pkg.dev/ml-pipeline/llm-text-embedding/tune-text-embedding-model/v1.1.1"

In [8]:
# Run the job. This will launch and then monitor the job until completion.
job = aiplatform.PipelineJob(
    display_name='tune-text-embedding',
    pipeline_root=PIPELINE_ROOT,
    template_path=template_uri,
    project=PROJECT_ID,
    location=LOCATION,
    parameter_values=PARAMS,
)

job.run()

Error when trying to get or create a GCS bucket for the pipeline output artifacts
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aiplatform/pipeline_jobs.py", line 398, in submit
    gcs_utils.create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist(
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aiplatform/utils/gcs_utils.py", line 235, in create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist
    pipelines_bucket = storage.Blob.from_string(
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/storage/blob.py", line 410, in from_string
    raise ValueError("URI scheme must be gs")
ValueError: URI scheme must be gs
Creating PipelineJob
PipelineJob created. Resource name: projects/255766800726/locations/us-central1/pipelineJobs/tune-text-embedding-model-20231128043632
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/255766800726/

In [9]:
# @title ## Define the training job and endpoint.
# @markdown Define the endpoint id if you'd like to use an existing endpoint. Otherwise leave blank and a new endpoint will be created.
# ex: projects/{PROJECT_NUMBER}/locations/{LOCATION}/endpoints/{ENDPOINT_NUMBER}
FULL_ENDPOINT_ID = ''  # @param {type:"string"}
# @markdown Define the job id if you'd like to use an existing job. If you'd like to use the submitted job from above this can be left blank.
# ex: projects/{PROJECT_NUMBER}/locations/{LOCATION}/pipelineJobs/{PIPELINE_NAME}
FULL_JOB_ID = ''  # @param {type:"string"}

if FULL_ENDPOINT_ID:
  custom_endpoint = aiplatform.Endpoint(
      FULL_ENDPOINT_ID, project=PROJECT_ID, location=LOCATION
  )
else:
  custom_endpoint = aiplatform.Endpoint.create(
      display_name='tuned_custom_embedding_endpoint',
      description='Endpoint for customized embeddings.',
      project=PROJECT_ID,
      location=LOCATION,
  )

if FULL_JOB_ID:
  job = aiplatform.PipelineJob.get(FULL_JOB_ID, project=PROJECT_ID)

Creating Endpoint
Create Endpoint backing LRO: projects/255766800726/locations/us-central1/endpoints/2477808826821115904/operations/1192826561150058496
Endpoint created. Resource name: projects/255766800726/locations/us-central1/endpoints/2477808826821115904
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/255766800726/locations/us-central1/endpoints/2477808826821115904')


In [10]:
# @markdown Define helper functions { display-mode: "form"}
def get_task_by_name(job: aiplatform.PipelineJob, task_name: str):
  for task in job.task_details:
    if task.task_name == task_name:
      return task
  raise ValueError(f'Task {task_name} not found')


def get_uploaded_model(
    job: aiplatform.PipelineJob, task_name: str = 'model-upload'
) -> aiplatform.Model:
  upload_task = get_task_by_name(job, task_name)
  upload_artifact = upload_task.outputs['model'].artifacts[0]
  return aiplatform.Model(upload_artifact.metadata['resourceName'])


def get_training_output_dir(
    job: aiplatform.PipelineJob, task_name: str = 'text-embedding-trainer'
) -> str:
  trainer_task = get_task_by_name(job, task_name)
  output_artifact = trainer_task.outputs['training_output'].artifacts[0]
  return output_artifact.uri


def get_df_from_jsonl(file_path: str, nrows: int | None = None) -> pd.DataFrame:
  with tf.io.gfile.GFile(file_path, 'r') as f:
    df = pd.read_json(f, lines=True, nrows=nrows)
  return df


def get_topk_scores(
    query_embedding: pd.DataFrame, corpus_embeddings: pd.DataFrame, k=10
) -> pd.DataFrame:
  similarity = corpus_embeddings.dot(query_embedding.T)
  topk_index = pd.DataFrame(
      {c: v.nlargest(n=k).index for c, v in similarity.items()}
  )
  return topk_index


def get_topk_documents(
    query_text: list[str],
    corpus_text: pd.DataFrame,
    corpus_embeddings: pd.DataFrame,
    k: int = 10,
) -> pd.DataFrame:
  response = custom_endpoint.predict(instances=query_text)
  query_embedding = np.asarray(response.predictions)
  topk = get_topk_scores(query_embedding, corpus_embeddings, k)
  return pd.DataFrame.from_dict(
      {
          query_text[c]: corpus_text.loc[v.values].values.ravel()
          for c, v in topk.items()
      },
      orient='columns',
  )

In [ ]:
# @title ## Deploy the model to the endpoint
INFERENCE_ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'  # @param {type:"string"}
INFERENCE_ACCELERATOR_COUNT = 1  # @param {type:"integer"}
INFERENCE_MACHINE_TYPE = 'a2-highgpu-1g'  # @param {type:"string"}

custom_model = get_uploaded_model(job)
custom_endpoint.deploy(
    custom_model,
    accelerator_type=INFERENCE_ACCELERATOR_TYPE,
    accelerator_count=INFERENCE_ACCELERATOR_COUNT,
    machine_type=INFERENCE_MACHINE_TYPE,
)

Deploying Model projects/255766800726/locations/us-central1/models/950978601979740160 to Endpoint : projects/255766800726/locations/us-central1/endpoints/2477808826821115904
Deploy Endpoint model backing LRO: projects/255766800726/locations/us-central1/endpoints/2477808826821115904/operations/4782195464164343808


In [ ]:
# @title ## Perform examples queries against the training corpus.

# Load the corpus embeddings from the training run.
training_output_dir = get_training_output_dir(job)
corpus_embeddings = get_df_from_jsonl(
    tf.io.gfile.join(training_output_dir, 'corpus_custom.jsonl')
).set_index('_id', drop=True)
corpus_text = get_df_from_jsonl(
    tf.io.gfile.join(training_output_dir, 'corpus_text.jsonl')
).set_index('_id', drop=True)

In [ ]:
# Define custom queries to see the output.

queries = [
    '''"Like something more" or "like something better"''',
    '''In what occasion could the word “precious” be taken disapprovingly, or sarcastically?''',
    '''Is the phrase "for free" correct?''',
]
output = get_topk_documents(queries, corpus_text, corpus_embeddings, k=10)

with pd.option_context('display.max_colwidth', 200):
  display(output)